In [1]:
import json
import evaluation_shared_task as evalfun

In [2]:
LANG = "en"

In [3]:
en_dev = json.load(open("data/Dev_"+LANG+"_new.json","rb"))

In [4]:
en_dev["text_splited"] = en_dev["text"].split(" ")

In [5]:
len(en_dev["text"])

278511

In [6]:
len(en_dev["begin_sentence"]),len(en_dev["end_sentence"])

(1384, 1384)

In [7]:
test3 = {}
test3["text"] = en_dev["text"]
test3["begin_sentence"] = []
test3["end_sentence"] = []
status = False
last_start = 0
start_log = 0
last_end = 0
end_log = 0
for i,word in enumerate(en_dev["text_splited"]):
    start_features = sum([
        (word[:1] == word[:1].upper() and word[:4] != word[:4].upper()),
        word == "-",
        word == "(",
        ( len(en_dev["text_splited"]) > i+1 and en_dev["text_splited"][i + 1] == ")"),
        word[:1] == "`"
    ])
    end_features = sum([
        word[-2:] == ".\n",
        word[-1:] == "." and\
        not ( len(en_dev["text_splited"]) > i+1 and en_dev["text_splited"][i + 1].isdigit() and\
             i>0 and en_dev["text_splited"][i - 1].isdigit()),
        word[-1:] == ";",
        word[-2:] == ";\n",
        word[-1:] == "\n",
        word[-2:] == ":\n",
    ])
    if start_features == 0 and end_features == 0:
        continue
    if start_features ==0:
        if status:
            test3["begin_sentence"].append(last_start)
            status = False
            end_log = end_features
            last_end = i
            start_log = 0
            last_start = 0
        else:
            if end_log < end_features:
                end_log = end_features
                last_end = i
        continue
    if end_features == 0:
        if status:
            if start_log < start_features:
                start_log = start_features
                last_start = i
        else:
            test3["end_sentence"].append(last_end)
            status = True
            last_start = i
            start_log = start_features
            end_log = 0
            last_end = 0
        continue
    if start_features != 0 and end_features != 0:
        if status:
            test3["begin_sentence"].append(last_start)
            status = False
            end_log = end_features
            last_end = i
            start_log = 0
            last_start = 0
            test3["end_sentence"].append(last_end)
            status = True
            last_start = i
            start_log = start_features
            end_log = 0
            last_end = 0
        else:
            test3["end_sentence"].append(last_end)
            status = True
            last_start = i
            start_log = start_features
            end_log = 0
            last_end = 0
            test3["begin_sentence"].append(last_start)
            status = False
            end_log = end_features
            last_end = i
            start_log = 0
            last_start = 0
        continue

In [8]:
evalfun.evaluate_result(en_dev,test3)

              precision    recall  f1-score   support

           O       0.99      0.98      0.98     47091
          BS       0.65      0.82      0.73      1384
          ES       0.71      0.94      0.81      1384

   micro avg       0.97      0.97      0.97     49859
   macro avg       0.79      0.91      0.84     49859
weighted avg       0.98      0.97      0.97     49859

